**Import libraries**

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 58.5 MB/s 
     |████████████████████████████████| 144 kB 61.9 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=cf56c61f946499e965fec07242e77ac54a29f98527c5873575268ffabe251d16
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import shutil
import math
import numpy as np
import pandas as pd
import wandb
from wandb.keras import WandbCallback

import keras
from keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, LSTM, GRU, Dense  
from tensorflow.keras.optimizers import Adam

**Downloading the dataset**

In [ ]:
!curl --header "Host: storage.googleapis.com" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36 Edg/89.0.774.77" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-US,en;q=0.9" --header "Referer: https://github.com/google-research-datasets/dakshina" "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar" -L -o "dakshina_dataset_v1.0.tar"
shutil.unpack_archive("/content/dakshina_dataset_v1.0.tar",'/content/')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0  67.8M      0  0:00:28  0:00:28 --:--:-- 90.7M


**Preparing the dataset**

In [ ]:
TRAIN_PATH = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
VAL_PATH = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
TEST_PATH = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"

# Function used to load data from a given path
def load_data(path):
  df = pd.read_csv(path, sep='\t', header=None)
  df.columns = ['Devanagari', 'Romanized', 'Attestations']
  df = df.dropna()
  input_texts = df['Romanized'].tolist()
  target_texts = df['Devanagari'].apply(lambda x: 'S' + x + 'E').tolist()
  return input_texts, target_texts

#loading training , testing and validation data
train_texts, train_target_texts = load_data(TRAIN_PATH)
val_texts, val_target_texts = load_data(VAL_PATH)
test_texts, test_target_texts = load_data(TEST_PATH)
print("Number of training samples: ", len(train_texts))
print("Number of validation samples: ", len(val_texts))
print("Number of testing samples: ", len(test_texts))


train_indices = np.arange(len(train_texts))
val_indices = np.arange(len(val_texts))
test_indices = np.arange(len(test_texts))

np.random.shuffle(train_indices)
np.random.shuffle(val_indices)

# Used to store vocabulary of source and target language
input_characters = set()
target_characters = set()

# Used to store texts after adding start and end token
train_target_texts_processed = []
val_target_texts_processed = []
test_target_texts_processed = []

# Adding starting and ending token in training data
for (input_text, target_text) in zip(train_texts, train_target_texts):
    # "S" -> start token, "E" -> end token, " " -> pad token
    target_text = "S" + target_text + "E"
    train_target_texts_processed.append(target_text)
    for char in input_text:
      input_characters.add(char)
    for char in target_text:
      target_characters.add(char)

# Adding starting and ending token in validation data
for (input_text, target_text) in zip(val_texts, val_target_texts):
    # "S" -> start token, "E" -> end token, " " -> pad token
    target_text = "S" + target_text + "E"
    val_target_texts_processed.append(target_text)
    for char in input_text:
      input_characters.add(char)
    for char in target_text:
      target_characters.add(char)

# Adding starting and ending token in testing data
for (input_text, target_text) in zip(test_texts, test_target_texts):
    # "S" -> start token, "E" -> end token, " " -> pad token
    target_text = "S" + target_text + "E"
    test_target_texts_processed.append(target_text)
    for char in input_text:
      input_characters.add(char)
    for char in target_text:
      target_characters.add(char)


input_texts = list(map(train_texts.__getitem__, train_indices))
target_texts = list(map(train_target_texts_processed.__getitem__, train_indices))

val_input_texts = list(map(val_texts.__getitem__, val_indices))
val_target_texts = list(map(val_target_texts_processed.__getitem__, val_indices))

test_input_texts = list(map(test_texts.__getitem__, test_indices))
test_target_texts = list(map(test_target_texts_processed.__getitem__, test_indices))

# Creating sorted vocabulary of source and target language
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# Add pad tokens
input_characters.insert(0, " ")
target_characters.insert(0, " ")

# Creating essential parameters
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(t) for t in input_texts])
max_decoder_seq_length = max([len(t) for t in target_texts])
val_max_encoder_seq_length = max([len(t) for t in val_input_texts])
val_max_decoder_seq_length = max([len(t) for t in val_target_texts])

test_max_encoder_seq_length = max([len(t) for t in test_input_texts])
test_max_decoder_seq_length = max([len(t) for t in test_target_texts])

# Mapping each character of vocabulary to index
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# defining shapes of input sequence of encoder after padding for training data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")

# defining shapes of input and target sequence of decoder after padding for training data
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Adding training data
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t+1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1: ] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# defining shapes of input sequence of encoder after padding for validation data
val_encoder_input_data = np.zeros((len(input_texts), val_max_encoder_seq_length), dtype="float32")

# defining shapes of input and target sequence of decoder after padding for validation data
val_decoder_input_data = np.zeros((len(input_texts), val_max_decoder_seq_length), dtype="float32")
val_decoder_target_data = np.zeros((len(input_texts), val_max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Adding validation data
for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1: ] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# defining shapes of input sequence of encoder after padding for testing data
test_encoder_input_data = np.zeros((len(input_texts), test_max_encoder_seq_length), dtype="float32")

# defining shapes of input and target sequence of decoder after padding for testing data
test_decoder_input_data = np.zeros((len(input_texts), test_max_decoder_seq_length), dtype="float32")
test_decoder_target_data = np.zeros((len(input_texts), test_max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Adding testing data
for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1: ] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# creating inverse map which maps integer to character
inverse_input_token_index = dict((i, char) for char, i in input_token_index.items())
inverse_target_token_index = dict((i, char) for char, i in target_token_index.items())

Number of training samples:  44202
Number of validation samples:  4358
Number of testing samples:  4502


In [ ]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for train inputs:", max_encoder_seq_length)
print("Max sequence length for train outputs:", max_decoder_seq_length)
print("Max sequence length for val inputs:", val_max_encoder_seq_length)
print("Max sequence length for val outputs:", val_max_decoder_seq_length)
print(input_characters)
print(target_characters)
print(input_token_index)
print(target_token_index)
print(encoder_input_data[10])
print(decoder_input_data[10])
print(decoder_target_data[10])

Number of samples: 44202
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for train inputs: 20
Max sequence length for train outputs: 23
Max sequence length for val inputs: 18
Max sequence length for val outputs: 18
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[' ', 'E', 'S', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']
{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{' ': 0, 'E': 1, 'S': 2, 'ँ': 3, 'ं': 4, 'ः

**Defining Transliteration Model**

In [ ]:
class TransliterationModel(object):
  def __init__(self, config):
    self.config = config

  def train_and_evaluate(self, encoder_input_data, decoder_input_data, decoder_target_data,
                         val_encoder_input_data, val_target_texts):
    # Encoder
    encoder_inputs = Input(shape=(None, ),name = 'Encoder_inputs')

    # Embedding layer: (num_encoder_tokens, input_embedding_size)
    encoder_embedded =  Embedding(num_encoder_tokens, self.config.input_embedding_size,
                         mask_zero=True, name='Encoder_embeddings')(encoder_inputs)
    encoder_outputs = encoder_embedded

    # Adding encoder layers and storing encoder states according to cell type
    if self.config.cell_type == 'RNN':
      encoder_layers = [SimpleRNN(self.config.hidden_units, 
                                  dropout=self.config.dropout, 
                                  return_sequences=True, 
                                  return_state=True, 
                                  name=f"Encoder_{layer_idx}")
                        for layer_idx in range(self.config.num_encoder_layers)]
      encoder_outputs, hidden = encoder_layers[0](encoder_outputs)
      encoder_states = [hidden]
      for layer_idx in range(1, self.config.num_encoder_layers):
        encoder_outputs, hidden = encoder_layers[layer_idx](encoder_outputs, initial_state=encoder_states)
        encoder_states = [hidden]  
    elif self.config.cell_type == 'LSTM':
      encoder_layers = [LSTM(self.config.hidden_units, 
                             dropout=self.config.dropout, 
                             return_sequences=True, 
                             return_state=True, 
                             name=f"Encoder_{layer_idx}")
                        for layer_idx in range(self.config.num_encoder_layers)]
      encoder_outputs, hidden, context = encoder_layers[0](encoder_outputs)
      encoder_states = [hidden, context]
      for layer_idx in range(1, self.config.num_encoder_layers):
        encoder_outputs, hidden, context = encoder_layers[layer_idx](encoder_outputs, initial_state=encoder_states)
        encoder_states = [hidden, context]
    elif self.config.cell_type == 'GRU':
      encoder_layers = [GRU(self.config.hidden_units, 
                            dropout=self.config.dropout, 
                            return_sequences=True, 
                            return_state=True, 
                            name=f"Encoder_{layer_idx}")
                        for layer_idx in range(self.config.num_encoder_layers)]
      encoder_outputs, hidden = encoder_layers[0](encoder_outputs)
      encoder_states = [hidden]
      for layer_idx in range(1, self.config.num_encoder_layers):
        encoder_outputs, hidden = encoder_layers[layer_idx](encoder_outputs, initial_state=encoder_states)
        encoder_states = [hidden]

    # Decoder
    decoder_inputs = Input(shape=(None,), name = 'Decoder_inputs')

    # Embedding layer: (num_decoder_tokens, hidden_units)
    decoder_embedded = Embedding(num_decoder_tokens, self.config.hidden_units,
                       mask_zero=True, name='Decoder_embeddings')(decoder_inputs)
    decoder_outputs = decoder_embedded

    # Adding decoder layers and storing decoder states according to cell type
    if self.config.cell_type == 'RNN':
      decoder_layers = [SimpleRNN(self.config.hidden_units, 
                                  dropout=self.config.dropout, 
                                  return_sequences=True, 
                                  return_state=True, 
                                  name=f"Decoder_{layer_idx}")
                        for layer_idx in range(self.config.num_decoder_layers)]
      decoder_outputs, _ = decoder_layers[0](decoder_outputs, initial_state=encoder_states)
      for layer_idx in range(1, self.config.num_decoder_layers):
        decoder_outputs, _ = decoder_layers[layer_idx](decoder_outputs, initial_state = encoder_states)
    if self.config.cell_type == 'LSTM':
      decoder_layers = [LSTM(self.config.hidden_units, 
                             dropout=self.config.dropout, 
                             return_sequences=True, 
                             return_state=True, 
                             name=f"Decoder_{layer_idx}")
                        for layer_idx in range(self.config.num_decoder_layers)]
      decoder_outputs, _, _ = decoder_layers[0](decoder_outputs, initial_state=encoder_states)
      for layer_idx in range(1, self.config.num_decoder_layers):
        decoder_outputs, _, _ = decoder_layers[layer_idx](decoder_outputs, initial_state = encoder_states)
    elif self.config.cell_type == 'GRU':
      decoder_layers = [GRU(self.config.hidden_units, 
                            dropout=self.config.dropout, 
                            return_sequences=True, 
                            return_state=True, 
                            name=f"Decoder_{layer_idx}")
                        for layer_idx in range(self.config.num_decoder_layers)]
      decoder_outputs, _ = decoder_layers[0](decoder_outputs, initial_state=encoder_states)
      for layer_idx in range(1, self.config.num_decoder_layers):
        decoder_outputs, _ = decoder_layers[layer_idx](decoder_outputs, initial_state=encoder_states)
    decoder_outputs = Dense(num_decoder_tokens, activation='softmax', name='dense')(decoder_outputs)

    # Defining our Seq2seq model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    optimizer = Adam(learning_rate=self.config.learning_rate, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer,
                  metrics=['accuracy'])
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=self.config.batch_size,
        epochs=self.config.epochs,
        validation_data=([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data),
        callbacks = [WandbCallback()]
    )
    
    model.save("transliteration_model")
    # model = keras.models.load_model("transliteration_model")
    
    # Wrap Encoder Decoder
    encoder_inputs = model.input[0]
    if self.config.cell_type in ['RNN', 'GRU']:
      encoder_outputs, hidden_state = model.get_layer(f'Encoder_{self.config.num_encoder_layers-1}').output
      encoder_states = [hidden_state]
      encoder = Model(encoder_inputs, encoder_states)
      decoder_inputs = model.input[1]
      decoder_outputs = model.get_layer('Decoder_embeddings')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []
      for i in range(self.config.num_decoder_layers):
        decoder_hidden = keras.Input(shape=(self.config.hidden_units,))
        states = [decoder_hidden]
        decoder_outputs, hidden_state_decoder = model.get_layer(f'Decoder_{i}')(decoder_outputs, initial_state=states)
        decoder_states += [hidden_state_decoder]
        decoder_states_inputs += states
    elif self.config.cell_type == 'LSTM':
      encoder_outputs, hidden_state, context_state = model.get_layer(f'Encoder_{self.config.num_encoder_layers-1}').output
      encoder_states = [hidden_state, context_state]
      encoder = Model(encoder_inputs, encoder_states)
      decoder_inputs = model.input[1]  # input_1
      decoder_outputs = model.get_layer('Decoder_embeddings')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []
      for i in range(self.config.num_decoder_layers):
        decoder_hidden = keras.Input(shape=(self.config.hidden_units,))
        decoder_context = keras.Input(shape=(self.config.hidden_units,))
        states = [decoder_hidden, decoder_context]
        decoder = model.get_layer(f'Decoder_{i}')
        decoder_outputs, hidden_state_decoder, context_state_decoder = decoder(decoder_outputs, initial_state=states)
        decoder_states += [hidden_state_decoder, context_state_decoder]
        decoder_states_inputs += states
    decoder_dense = model.get_layer('dense')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
 
    # finding validation accuracy and logging it into wandb
    total, correct = 0, 0
    output_list, target_list = [], []
    for i in range(len(val_texts)):
      output = self.decode_to_text(val_encoder_input_data[i:i+1], encoder, decoder)
      target = val_target_texts[i][1:len(val_target_texts[i])-1]
      output = output[0:len(output)-1]
      output = output.replace(' ', '').replace('S', '').replace('E', '')
      target = target.replace(' ', '').replace('S', '').replace('E', '')
      output_list.append(output)
      target_list.append(target)
      # print('O/P TGT: ', output, target)
      if output == target:
        correct += 1
      total += 1
      validation_word_accuracy = correct / total
      if i % 100 == 0:
        print({o: t for o, t in zip(output_list[i:i+100], target_list[i:i+100])})
        wandb.log({'validation_word_accuracy' : validation_word_accuracy})
    word_val_accuracy = correct / total
    wandb.log({'word_val_accuracy' : word_val_accuracy})
      
  def decode_to_text(self, inputs, encoder, decoder):
    encoder_states = [encoder.predict(inputs) for _ in range(self.config.num_decoder_layers)]
    target = np.array([[target_token_index['S']]])
    sentence, done = "", False
    beam_width = 1 if self.config.decoding_strategy == 'greedy' else self.config.beam_width
    sentence = self.beam_search_decoder(inputs, encoder, decoder, beam_width)
    return sentence


  def beam_search_decoder(self, inputs, encoder, decoder, beam_width):
        
        done, decoded_sentence = False, ""

        # Get encoder states
        encoder_states = [encoder.predict(inputs) for _ in range(self.config.num_decoder_layers)]

        # Decoder input begins with Start Token "S"
        target_sequence = np.array([[target_token_index["S"]]])

        # sum_of_log_probs (score), flag for end of current sequence, target_sequence, states , sequence_token, sequence_char
        sequences = [[0.0, 0,  target_sequence, encoder_states,  list(),list()]]
        while not done:
            candidates = list()
            for i in range(len(sequences)):
              output = decoder.predict([sequences[i][2]] + sequences[i][3])
              output_tokens, states = output[0], output[1:]
              prob = output_tokens[0,-1,:]
              
              score, flag, _, _, sequence_token, sequence_char = sequences[i]
              
              if flag == 0:
                for j in range(len(inverse_target_token_index)):
                  char = inverse_target_token_index[j]
                  target_sequence = np.array([[j]])
                  candidate = [score - np.log(prob[j]), 0, target_sequence, states,  sequence_token + [j] , sequence_char + [char] ]
                  candidates.append(candidate)
            sorted_candidates = sorted(candidates, key=lambda x:x[0])
            k = min(beam_width, len(sorted_candidates))
            sequences = sorted_candidates[:k]
            done = True
           
            for sequence in range(len(sequences)):
                score, flag, tgt_seq, states, sequence_token, sequence_char = sequences[sequence]
                if (len(sequence_char) > max_decoder_seq_length) or (sequence_char[-1] == "E"): 
                  flag = 1
                sequences[sequence][1] = flag
                done = False if flag == 0 else done
            if sequences[0][-1][-1]=="E": 
              done = True
        top_decoded_sentence = ''.join(sequences[0][5])
        return top_decoded_sentence

**Defining the Sweep Hyperparameter dictionary**

In [ ]:
wandb.login()
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'word_val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {

        'dropout': {
            'values': [0, 0.2, 0.3]
        },
        'learning_rate': {
            'values': [0.001,0.0001,0.0003,0.0005]
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'input_embedding_size': {
            'values': [32, 64, 128, 256, 512]
        },
        'num_encoder_layers': {
            'values': [1, 2, 3, 4]
        },
        'num_decoder_layers': {
            'values': [1, 2, 3, 4]
        },
        'hidden_size':{
            'values': [32, 64, 128, 256, 512]
        },
        'cell_type': {
            'values': ['RNN', 'LSTM', 'GRU']
        },
        'decoding_strategy': {
            'values': ['beam_search', 'greedy']
        },
        'beam_sizes':{
            'values': [3, 5, 7]
        },
        'epochs':{
            'values': [5,10,15,20]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="Assigment3_Q2", entity="cs21s006_cs21s043")
print('sweep_id: ', sweep_id)

Create sweep with ID: alseo6jh
Sweep URL: https://wandb.ai/cs21s006_cs21s043/Assigment3_Q2/sweeps/alseo6jh
sweep_id:  alseo6jh


In [ ]:
def spawn_fn():
      with wandb.init(project="Assignment3_Q2", entity="cs21s006_cs21s043") as run:
        config = wandb.config
        wandb.run.name = 'ep-'+str(config.epochs)+'-dr-'+str(config.dropout)+'-lr-'+str(config.learning_rate)+'-bs'+str(config.batch_size)+'-es-'+str(config.input_embedding_size)\
        +'-el-'+str(config.num_encoder_layers)+'-dl-'+str(config.num_decoder_layers)+'-hs-'+str(config.hidden_size)+'-cell-'+str(config.cell_type)+'-dec_str-'+str(config.decoding_strategy)\
        +'-bs-'+str(config.beam_sizes)

        model_transliteration = TransliterationModel(config)
        model_transliteration.train_and_evaluate(encoder_input_data,decoder_input_data,decoder_target_data,val_encoder_input_data, val_target_texts)

**By this cell only 15 sweeps ran**

In [ ]:
wandb.agent("ajlstr1d", function = spawn_fn , count=50)

wandb: Agent Starting Run: 3khcbuep with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 5
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	hidden_size: 256
wandb: 	input_embedding_size: 64
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/5
346/346 [==============================] - 28s 52ms/step - loss: 1.0185 - accuracy: 0.3388 - val_loss: 0.1065 - val_accuracy: 0.4026 - _timestamp: 1650975228.0000 - _runtime: 37.0000
Epoch 2/5
346/346 [==============================] - 12s 35ms/step - loss: 0.7638 - accuracy: 0.4587 - val_loss: 0.0832 - val_accuracy: 0.5168 - _timestamp: 1650975240.0000 - _runtime: 49.0000
Epoch 3/5
346/346 [==============================] - 12s 35ms/step - loss: 0.5834 - accuracy: 0.5721 - val_loss: 0.0619 - val_accuracy: 0.6352 - _timestamp: 1650975253.0000 - _runtime: 62.0000
Epoch 4/5
346/346 [==============================] - 12s 35ms/step - loss: 0.4514 - accuracy: 0.6598 - val_loss: 0.0478 - val_accuracy: 0.7094 - _timestamp: 1650975265.0000 - _runtime: 74.0000
Epoch 5/5
346/346 [==============================] - 12s 35ms/step - loss: 0.3677 - accuracy: 0.7173 - val_loss: 0.0405 - val_accuracy: 0.7488 - _timestamp: 1650975277.0000 - _runtime: 86.0000


accuracy,▁▃▅▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▃▆▇█
val_loss,█▆▃▂▁
validation_word_accuracy,▄▃▁▄▅▇▇▆▇▇█████▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅
word_val_accuracy,▁
accuracy,0.71727
best_epoch,4
best_val_loss,0.04051
epoch,4


wandb: Agent Starting Run: wrva5utm with config:
wandb: 	batch_size: 256
wandb: 	beam_sizes: 5
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.3
wandb: 	epochs: 20
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 32
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
173/173 [==============================] - 42s 117ms/step - loss: 1.1726 - accuracy: 0.2268 - val_loss: 0.1278 - val_accuracy: 0.3234 - _timestamp: 1650977135.0000 - _runtime: 56.0000
Epoch 2/20
173/173 [==============================] - 13s 77ms/step - loss: 0.9293 - accuracy: 0.3641 - val_loss: 0.1057 - val_accuracy: 0.4008 - _timestamp: 1650977149.0000 - _runtime: 70.0000
Epoch 3/20
173/173 [==============================] - 13s 77ms/step - loss: 0.7871 - accuracy: 0.4370 - val_loss: 0.0867 - val_accuracy: 0.4911 - _timestamp: 1650977162.0000 - _runtime: 83.0000
Epoch 4/20
173/173 [==============================] - 13s 77ms/step - loss: 0.6771 - accuracy: 0.5020 - val_loss: 0.0744 - val_accuracy: 0.5533 - _timestamp: 1650977175.0000 - _runtime: 96.0000
Epoch 5/20
173/173 [==============================] - 13s 76ms/step - loss: 0.5807 - accuracy: 0.5628 - val_loss: 0.0630 - val_accuracy: 0.6207 - _timestamp: 1650977188.0000 - _runtime: 109.0000
Epoch 6/20
173/173 [========

...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lskdcqfk with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 3
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	input_embedding_size: 32
wandb: 	learning_rate: 0.0001
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/10
346/346 [==============================] - 41s 69ms/step - loss: 1.2762 - accuracy: 0.1970 - val_loss: 0.1424 - val_accuracy: 0.2379 - _timestamp: 1651003251.0000 - _runtime: 55.0000
Epoch 2/10
346/346 [==============================] - 18s 53ms/step - loss: 1.1191 - accuracy: 0.2577 - val_loss: 0.1334 - val_accuracy: 0.3112 - _timestamp: 1651003269.0000 - _runtime: 73.0000
Epoch 3/10
346/346 [==============================] - 18s 52ms/step - loss: 1.0418 - accuracy: 0.3269 - val_loss: 0.1260 - val_accuracy: 0.3404 - _timestamp: 1651003287.0000 - _runtime: 91.0000
Epoch 4/10
346/346 [==============================] - 18s 53ms/step - loss: 0.9922 - accuracy: 0.3498 - val_loss: 0.1190 - val_accuracy: 0.3598 - _timestamp: 1651003306.0000 - _runtime: 110.0000
Epoch 5/10
346/346 [==============================] - 18s 52ms/step - loss: 0.9413 - accuracy: 0.3651 - val_loss: 0.1145 - val_accuracy: 0.3702 - _timestamp: 1651003323.0000 - _runtime: 127.0000
Epoch 6/10
346/346 [========

accuracy,▁▃▅▆▇▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▁▁▁
val_accuracy,▁▄▅▆▆▇▇▇██
val_loss,█▆▅▄▃▂▂▂▁▁
validation_word_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▇▇▇▇▇▇▆▆▆
word_val_accuracy,▁
accuracy,0.40398
best_epoch,9
best_val_loss,0.10331
epoch,9


wandb: Agent Starting Run: vkioczb9 with config:
wandb: 	batch_size: 64
wandb: 	beam_sizes: 5
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: greedy
wandb: 	dropout: 0.3
wandb: 	epochs: 15
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/15
691/691 [==============================] - 58s 59ms/step - loss: 0.7142 - accuracy: 0.4959 - val_loss: 0.0432 - val_accuracy: 0.7321 - _timestamp: 1651005439.0000 - _runtime: 72.0000
Epoch 2/15
691/691 [==============================] - 35s 50ms/step - loss: 0.2760 - accuracy: 0.7812 - val_loss: 0.0285 - val_accuracy: 0.8179 - _timestamp: 1651005473.0000 - _runtime: 106.0000
Epoch 3/15
691/691 [==============================] - 34s 50ms/step - loss: 0.1877 - accuracy: 0.8498 - val_loss: 0.0237 - val_accuracy: 0.8460 - _timestamp: 1651005508.0000 - _runtime: 141.0000
Epoch 4/15
691/691 [==============================] - 34s 50ms/step - loss: 0.1445 - accuracy: 0.8844 - val_loss: 0.0219 - val_accuracy: 0.8558 - _timestamp: 1651005542.0000 - _runtime: 175.0000
Epoch 5/15
691/691 [==============================] - 34s 50ms/step - loss: 0.1160 - accuracy: 0.9073 - val_loss: 0.0209 - val_accuracy: 0.8655 - _timestamp: 1651005576.0000 - _runtime: 209.0000
Epoch 6/15
691/691 [======

In [ ]:
wandb.agent("ajlstr1d", function = spawn_fn , count=20)

wandb: Agent Starting Run: n3t3hivm with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 7
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1
wandb: Currently logged in as: cs21s043 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/5
346/346 [==============================] - 40s 62ms/step - loss: 0.8565 - accuracy: 0.4143 - val_loss: 0.0584 - val_accuracy: 0.6417 - _timestamp: 1651009752.0000 - _runtime: 53.0000
Epoch 2/5
346/346 [==============================] - 16s 47ms/step - loss: 0.3582 - accuracy: 0.7210 - val_loss: 0.0344 - val_accuracy: 0.7839 - _timestamp: 1651009768.0000 - _runtime: 69.0000
Epoch 3/5
346/346 [==============================] - 16s 46ms/step - loss: 0.2335 - accuracy: 0.8147 - val_loss: 0.0273 - val_accuracy: 0.8221 - _timestamp: 1651009784.0000 - _runtime: 85.0000
Epoch 4/5
346/346 [==============================] - 16s 46ms/step - loss: 0.1776 - accuracy: 0.8586 - val_loss: 0.0235 - val_accuracy: 0.8466 - _timestamp: 1651009800.0000 - _runtime: 101.0000
Epoch 5/5
346/346 [==============================] - 16s 46ms/step - loss: 0.1435 - accuracy: 0.8858 - val_loss: 0.0216 - val_accuracy: 0.8576 - _timestamp: 1651009816.0000 - _runtime: 117.0000


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▆▇██
val_loss,█▃▂▁▁
validation_word_accuracy,▁█▄▇▅▅▅▅▅▄▃▃▄▅▅▅▅▆▅▅▅▅▅▅▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅
word_val_accuracy,▁
accuracy,0.88585
best_epoch,4
best_val_loss,0.02157
epoch,4


wandb: Agent Starting Run: b3lhdphv with config:
wandb: 	batch_size: 256
wandb: 	beam_sizes: 7
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.3
wandb: 	epochs: 15
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 512
wandb: 	learning_rate: 0.0003
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/15
173/173 [==============================] - 29s 82ms/step - loss: 1.1498 - accuracy: 0.2480 - val_loss: 0.1151 - val_accuracy: 0.3837 - _timestamp: 1651011977.0000 - _runtime: 39.0000
Epoch 2/15
173/173 [==============================] - 9s 55ms/step - loss: 0.7215 - accuracy: 0.4826 - val_loss: 0.0678 - val_accuracy: 0.5794 - _timestamp: 1651011986.0000 - _runtime: 48.0000
Epoch 3/15
173/173 [==============================] - 9s 54ms/step - loss: 0.4834 - accuracy: 0.6257 - val_loss: 0.0501 - val_accuracy: 0.6854 - _timestamp: 1651011995.0000 - _runtime: 57.0000
Epoch 4/15
173/173 [==============================] - 9s 54ms/step - loss: 0.3701 - accuracy: 0.7109 - val_loss: 0.0398 - val_accuracy: 0.7467 - _timestamp: 1651012005.0000 - _runtime: 67.0000
Epoch 5/15
173/173 [==============================] - 9s 54ms/step - loss: 0.3009 - accuracy: 0.7621 - val_loss: 0.0338 - val_accuracy: 0.7842 - _timestamp: 1651012014.0000 - _runtime: 76.0000
Epoch 6/15
173/173 [==============

...

Epoch 1/10
346/346 [==============================] - 33s 57ms/step - loss: 0.8204 - accuracy: 0.4424 - val_loss: 0.0570 - val_accuracy: 0.6578 - _timestamp: 1651053739.0000 - _runtime: 45.0000
Epoch 2/10
346/346 [==============================] - 15s 44ms/step - loss: 0.3141 - accuracy: 0.7558 - val_loss: 0.0304 - val_accuracy: 0.8080 - _timestamp: 1651053754.0000 - _runtime: 60.0000
Epoch 3/10
346/346 [==============================] - 15s 44ms/step - loss: 0.1944 - accuracy: 0.8460 - val_loss: 0.0245 - val_accuracy: 0.8444 - _timestamp: 1651053769.0000 - _runtime: 75.0000
Epoch 4/10
346/346 [==============================] - 15s 44ms/step - loss: 0.1475 - accuracy: 0.8824 - val_loss: 0.0222 - val_accuracy: 0.8556 - _timestamp: 1651053784.0000 - _runtime: 90.0000
Epoch 5/10
346/346 [==============================] - 15s 45ms/step - loss: 0.1166 - accuracy: 0.9078 - val_loss: 0.0213 - val_accuracy: 0.8622 - _timestamp: 1651053800.0000 - _runtime: 106.0000
Epoch 6/10
346/346 [=========

accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
val_accuracy,▁▆▇▇██████
val_loss,█▃▂▁▁▁▁▁▁▁
validation_word_accuracy,▁▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████▇▇▇
word_val_accuracy,▁
accuracy,0.96859
best_epoch,6
best_val_loss,0.0201
epoch,9


wandb: Agent Starting Run: 42iy9gd7 with config:
wandb: 	batch_size: 256
wandb: 	beam_sizes: 7
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.2
wandb: 	epochs: 20
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 256
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
173/173 [==============================] - 35s 97ms/step - loss: 1.1360 - accuracy: 0.2527 - val_loss: 0.1125 - val_accuracy: 0.3959 - _timestamp: 1651055691.0000 - _runtime: 47.0000
Epoch 2/20
173/173 [==============================] - 11s 64ms/step - loss: 0.6820 - accuracy: 0.5113 - val_loss: 0.0616 - val_accuracy: 0.6238 - _timestamp: 1651055702.0000 - _runtime: 58.0000
Epoch 3/20
173/173 [==============================] - 11s 63ms/step - loss: 0.4237 - accuracy: 0.6745 - val_loss: 0.0428 - val_accuracy: 0.7358 - _timestamp: 1651055713.0000 - _runtime: 69.0000
Epoch 4/20
173/173 [==============================] - 11s 63ms/step - loss: 0.3116 - accuracy: 0.7558 - val_loss: 0.0350 - val_accuracy: 0.7807 - _timestamp: 1651055724.0000 - _runtime: 80.0000
Epoch 5/20
173/173 [==============================] - 11s 64ms/step - loss: 0.2480 - accuracy: 0.8043 - val_loss: 0.0291 - val_accuracy: 0.8153 - _timestamp: 1651055735.0000 - _runtime: 91.0000
Epoch 6/20
173/173 [==========

accuracy,▁▄▅▆▆▇▇▇▇▇██████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▇▇▇██████████████
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_word_accuracy,▁██▇▆▆▆▆▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆
word_val_accuracy,▁
accuracy,0.96683
best_epoch,12
best_val_loss,0.0211
epoch,19


In [ ]:
wandb.agent("ajlstr1d", function = spawn_fn , count=20)

wandb: Agent Starting Run: 182sh6tz with config:
wandb: 	batch_size: 256
wandb: 	beam_sizes: 3
wandb: 	cell_type: LSTM
wandb: 	decoding_strategy: greedy
wandb: 	dropout: 0.3
wandb: 	epochs: 20
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 64
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
173/173 [==============================] - 28s 80ms/step - loss: 1.1457 - accuracy: 0.2552 - val_loss: 0.1196 - val_accuracy: 0.3466 - _timestamp: 1651063607.0000 - _runtime: 41.0000
Epoch 2/20
173/173 [==============================] - 9s 53ms/step - loss: 0.8808 - accuracy: 0.3899 - val_loss: 0.0975 - val_accuracy: 0.4441 - _timestamp: 1651063616.0000 - _runtime: 50.0000
Epoch 3/20
173/173 [==============================] - 9s 53ms/step - loss: 0.6924 - accuracy: 0.5051 - val_loss: 0.0710 - val_accuracy: 0.5866 - _timestamp: 1651063626.0000 - _runtime: 60.0000
Epoch 4/20
173/173 [==============================] - 9s 52ms/step - loss: 0.5196 - accuracy: 0.6150 - val_loss: 0.0528 - val_accuracy: 0.6837 - _timestamp: 1651063635.0000 - _runtime: 69.0000
Epoch 5/20
173/173 [==============================] - 9s 52ms/step - loss: 0.4025 - accuracy: 0.6938 - val_loss: 0.0410 - val_accuracy: 0.7481 - _timestamp: 1651063644.0000 - _runtime: 78.0000
Epoch 6/20
173/173 [==============

accuracy,▁▂▄▅▆▆▆▇▇▇▇▇▇███████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▅▆▇▇▇████████████
val_loss,█▆▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
validation_word_accuracy,▁▇▄▆▄▄▆▆▅▆▅▅▆▆▆▆▇▇▇▇██▇██▇▇▇██████████▇█
word_val_accuracy,▁
accuracy,0.93276
best_epoch,19
best_val_loss,0.02005
epoch,19


wandb: Agent Starting Run: ied1rsb5 with config:
wandb: 	batch_size: 256
wandb: 	beam_sizes: 5
wandb: 	cell_type: GRU
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.2
wandb: 	epochs: 20
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 256
wandb: 	learning_rate: 0.0003
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
173/173 [==============================] - 16s 42ms/step - loss: 1.1043 - accuracy: 0.3211 - val_loss: 0.1136 - val_accuracy: 0.3750 - _timestamp: 1651065954.0000 - _runtime: 25.0000
Epoch 2/20
173/173 [==============================] - 5s 28ms/step - loss: 0.8649 - accuracy: 0.3974 - val_loss: 0.1026 - val_accuracy: 0.4180 - _timestamp: 1651065959.0000 - _runtime: 30.0000
Epoch 3/20
173/173 [==============================] - 5s 29ms/step - loss: 0.7717 - accuracy: 0.4558 - val_loss: 0.0892 - val_accuracy: 0.4969 - _timestamp: 1651065964.0000 - _runtime: 35.0000
Epoch 4/20
173/173 [==============================] - 5s 29ms/step - loss: 0.6456 - accuracy: 0.5411 - val_loss: 0.0707 - val_accuracy: 0.5953 - _timestamp: 1651065969.0000 - _runtime: 40.0000
Epoch 5/20
173/173 [==============================] - 5s 29ms/step - loss: 0.5029 - accuracy: 0.6332 - val_loss: 0.0546 - val_accuracy: 0.6779 - _timestamp: 1651065974.0000 - _runtime: 45.0000
Epoch 6/20
173/173 [==============

...

wandb: Agent Starting Run: 6dgfj91b with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 3
wandb: 	cell_type: GRU
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 128
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/5
346/346 [==============================] - 42s 73ms/step - loss: 0.9641 - accuracy: 0.3504 - val_loss: 0.0916 - val_accuracy: 0.4812 - _timestamp: 1651092073.0000 - _runtime: 56.0000
Epoch 2/5
346/346 [==============================] - 19s 55ms/step - loss: 0.5469 - accuracy: 0.5948 - val_loss: 0.0480 - val_accuracy: 0.7099 - _timestamp: 1651092092.0000 - _runtime: 75.0000
Epoch 3/5
346/346 [==============================] - 19s 55ms/step - loss: 0.3061 - accuracy: 0.7623 - val_loss: 0.0334 - val_accuracy: 0.7931 - _timestamp: 1651092111.0000 - _runtime: 94.0000
Epoch 4/5
346/346 [==============================] - 19s 55ms/step - loss: 0.2171 - accuracy: 0.8296 - val_loss: 0.0273 - val_accuracy: 0.8257 - _timestamp: 1651092130.0000 - _runtime: 113.0000
Epoch 5/5
346/346 [==============================] - 19s 55ms/step - loss: 0.1707 - accuracy: 0.8657 - val_loss: 0.0248 - val_accuracy: 0.8438 - _timestamp: 1651092149.0000 - _runtime: 132.0000


accuracy,▁▄▇██
epoch,▁▃▅▆█
loss,█▄▂▁▁
val_accuracy,▁▅▇██
val_loss,█▃▂▁▁
validation_word_accuracy,▁▇▄█▆▇▅▆▆▅▄▄▄▅▄▅▅▆▇▆▇██▇▇██████▇▇██▇▇▇▆▆
word_val_accuracy,▁
accuracy,0.86567
best_epoch,4
best_val_loss,0.02477
epoch,4


wandb: Agent Starting Run: 3zp3srti with config:
wandb: 	batch_size: 64
wandb: 	beam_sizes: 3
wandb: 	cell_type: GRU
wandb: 	decoding_strategy: greedy
wandb: 	dropout: 0.2
wandb: 	epochs: 20
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
691/691 [==============================] - 28s 28ms/step - loss: 0.6815 - accuracy: 0.5315 - val_loss: 0.0406 - val_accuracy: 0.7489 - _timestamp: 1651094142.0000 - _runtime: 40.0000
Epoch 2/20
691/691 [==============================] - 17s 25ms/step - loss: 0.2452 - accuracy: 0.8070 - val_loss: 0.0259 - val_accuracy: 0.8335 - _timestamp: 1651094160.0000 - _runtime: 58.0000
Epoch 3/20
688/691 [============================>.] - ETA: 0s - loss: 0.1682 - accuracy: 0.8655

In [ ]:
wandb.agent("96qio1zu", function = spawn_fn , count=20)

wandb: Agent Starting Run: yux58zfg with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 7
wandb: 	cell_type: GRU
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.3
wandb: 	epochs: 30
wandb: 	hidden_size: 512
wandb: 	input_embedding_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/30
346/346 [==============================] - 40s 63ms/step - loss: 0.7896 - accuracy: 0.4558 - val_loss: 0.0484 - val_accuracy: 0.7057 - _timestamp: 1651103106.0000 - _runtime: 52.0000
Epoch 2/30
346/346 [==============================] - 16s 47ms/step - loss: 0.2984 - accuracy: 0.7659 - val_loss: 0.0270 - val_accuracy: 0.8283 - _timestamp: 1651103122.0000 - _runtime: 68.0000
Epoch 3/30
346/346 [==============================] - 16s 47ms/step - loss: 0.1890 - accuracy: 0.8485 - val_loss: 0.0224 - val_accuracy: 0.8567 - _timestamp: 1651103138.0000 - _runtime: 84.0000
Epoch 4/30
346/346 [==============================] - 16s 47ms/step - loss: 0.1456 - accuracy: 0.8818 - val_loss: 0.0203 - val_accuracy: 0.8700 - _timestamp: 1651103155.0000 - _runtime: 101.0000
Epoch 5/30
346/346 [==============================] - 16s 47ms/step - loss: 0.1184 - accuracy: 0.9034 - val_loss: 0.0199 - val_accuracy: 0.8724 - _timestamp: 1651103171.0000 - _runtime: 117.0000
Epoch 6/30
346/346 [========

accuracy,▁▅▆▇▇▇▇███████████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇███████████████████████████
val_loss,█▃▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃
validation_word_accuracy,▁▁▄▃▆▆▆█▇█▇███▇████▇▇▇▇▇██▇▇▇▇▇▇██▇▇▇▇▇▇
word_val_accuracy,▁
accuracy,0.97291
best_epoch,5
best_val_loss,0.01962
epoch,29


wandb: Agent Starting Run: 7huyo8sj with config:
wandb: 	batch_size: 64
wandb: 	beam_sizes: 5
wandb: 	cell_type: RNN
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.2
wandb: 	epochs: 30
wandb: 	hidden_size: 256
wandb: 	input_embedding_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/30
691/691 [==============================] - 79s 109ms/step - loss: 1.0345 - accuracy: 0.3342 - val_loss: 0.1074 - val_accuracy: 0.4210 - _timestamp: 1651105743.0000 - _runtime: 87.0000
Epoch 2/30
691/691 [==============================] - 73s 105ms/step - loss: 0.7934 - accuracy: 0.4603 - val_loss: 0.0883 - val_accuracy: 0.5125 - _timestamp: 1651105815.0000 - _runtime: 159.0000
Epoch 3/30
691/691 [==============================] - 73s 105ms/step - loss: 0.6790 - accuracy: 0.5278 - val_loss: 0.0749 - val_accuracy: 0.5772 - _timestamp: 1651105888.0000 - _runtime: 232.0000
Epoch 4/30
691/691 [==============================] - 72s 105ms/step - loss: 0.5915 - accuracy: 0.5780 - val_loss: 0.0651 - val_accuracy: 0.6277 - _timestamp: 1651105960.0000 - _runtime: 304.0000
Epoch 5/30
691/691 [==============================] - 73s 105ms/step - loss: 0.5285 - accuracy: 0.6142 - val_loss: 0.0582 - val_accuracy: 0.6549 - _timestamp: 1651106033.0000 - _runtime: 377.0000
Epoch 6/30
691/691 [=

...

Epoch 1/35
691/691 [==============================] - 65s 62ms/step - loss: 0.5577 - accuracy: 0.5993 - val_loss: 0.0304 - val_accuracy: 0.8055 - _timestamp: 1651153742.0000 - _runtime: 78.0000
Epoch 2/35
691/691 [==============================] - 36s 52ms/step - loss: 0.2037 - accuracy: 0.8352 - val_loss: 0.0226 - val_accuracy: 0.8540 - _timestamp: 1651153778.0000 - _runtime: 114.0000
Epoch 3/35
691/691 [==============================] - 36s 52ms/step - loss: 0.1461 - accuracy: 0.8798 - val_loss: 0.0209 - val_accuracy: 0.8669 - _timestamp: 1651153814.0000 - _runtime: 150.0000
Epoch 4/35
691/691 [==============================] - 36s 52ms/step - loss: 0.1168 - accuracy: 0.9041 - val_loss: 0.0204 - val_accuracy: 0.8736 - _timestamp: 1651153850.0000 - _runtime: 186.0000
Epoch 5/35
691/691 [==============================] - 36s 52ms/step - loss: 0.0987 - accuracy: 0.9183 - val_loss: 0.0200 - val_accuracy: 0.8762 - _timestamp: 1651153886.0000 - _runtime: 222.0000
Epoch 6/35
691/691 [======

accuracy,▁▆▇▇▇▇█████████████████████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇▇███████▇██▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_loss,█▃▂▁▁▁▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▆▆▆▆
validation_word_accuracy,▁▇█▃▅▄▃▄▅▅▄▄▄▅▄▅▅▅▅▅▅▅▅▅▅▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅
word_val_accuracy,▁
accuracy,0.94589
best_epoch,4
best_val_loss,0.02003
epoch,34


wandb: Agent Starting Run: k9r60n8i with config:
wandb: 	batch_size: 128
wandb: 	beam_sizes: 3
wandb: 	cell_type: GRU
wandb: 	decoding_strategy: beam_search
wandb: 	dropout: 0.3
wandb: 	epochs: 30
wandb: 	hidden_size: 256
wandb: 	input_embedding_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/30
346/346 [==============================] - 47s 71ms/step - loss: 1.0147 - accuracy: 0.3188 - val_loss: 0.1028 - val_accuracy: 0.4134 - _timestamp: 1651157286.0000 - _runtime: 59.0000
Epoch 2/30
346/346 [==============================] - 18s 51ms/step - loss: 0.6854 - accuracy: 0.5045 - val_loss: 0.0634 - val_accuracy: 0.6249 - _timestamp: 1651157303.0000 - _runtime: 76.0000
Epoch 3/30
346/346 [==============================] - 18s 51ms/step - loss: 0.4705 - accuracy: 0.6444 - val_loss: 0.0434 - val_accuracy: 0.7335 - _timestamp: 1651157321.0000 - _runtime: 94.0000
Epoch 4/30
346/346 [==============================] - 18s 51ms/step - loss: 0.3533 - accuracy: 0.7251 - val_loss: 0.0345 - val_accuracy: 0.7864 - _timestamp: 1651157339.0000 - _runtime: 112.0000
Epoch 5/30
346/346 [==============================] - 17s 51ms/step - loss: 0.2838 - accuracy: 0.7772 - val_loss: 0.0291 - val_accuracy: 0.8192 - _timestamp: 1651157356.0000 - _runtime: 129.0000
Epoch 6/30
346/346 [========

accuracy,▁▃▅▆▆▆▇▇▇▇▇▇▇▇████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▇▇▇████████████████████████
val_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_word_accuracy,▂▆▃▁▅▅▅▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆
word_val_accuracy,▁
accuracy,0.94633
best_epoch,13
best_val_loss,0.02068
epoch,29
